In [91]:
import pandas as pd
import numpy as np
import glob
import re
%matplotlib inline
import bs4
from bs4 import BeautifulSoup
import csv

In [92]:
# Open data and convert it to pandas
data=[]
with open('../data/class-data-with-all-tests-and-concept-check-NO_IDENTIFIERS.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    data = list(csv_reader)
    
headers = data[0]
rest = data[1:]

everything = pd.DataFrame(rest, columns=headers)

In [93]:
def as_int(dat):
    return dat.apply(lambda x: int(x) if x else 0)
def score1a(data):
    return (data['1a'] == 'c').astype(int)
def score1b(data):
    a,b,c,d,e,f = as_int(data['1b.a']),as_int(data['1b.b']),as_int(data['1b.c']),as_int(data['1b.d']),as_int(data['1b.e']),as_int(data['1b.f'])
    return (a+(1-b)+c+(1-d)+e+(1-f))/6
def score2a(data):
    return (data['2a'] == 'a').astype(int)
def score2b(data):
    return (data['2b'] == '0').astype(int)
def score2c(data):
    a,b,c,d,e = map(as_int, (data['2c.a'],data['2c.b'],data['2c.c'],data['2c.d'],data['2c.e']))
    return ((1-a)+b+(1-c)+(1-d)+(1-e))/5
def score3(data):
    a,b,c,d,e,f = map(as_int, (data['3.a'],data['3.b'],data['3.c'],data['3.d'],data['3.e'],data['3.f']))
    return ((1-a)+b+c+d+(1-e)+f)/6
def score4a(data):
    return (data['4a'] == 'd').astype(int)
def score5(data):
    a,b,c,d,e = map(as_int, (data['5.a'],data['5.b'],data['5.c'],data['5.d'],data['5.e']))
    return (a+(1-b)+(1-c)+d+(1-e))/5
def score6a(data):
    return (data['6a'] == '1').astype(int)
def score6b(data):
    return (data['6b'] == 'c').astype(int)
def score7(data):
    return (data['7'] == 'd').astype(int)
def score8(data):
    a,b,c,d,e,f = map(as_int, (data['8.a'],data['8.b'],data['8.c'],data['8.d'],data['8.e'],data['8.f']))
    return ((1-a)+(1-b)+(1-c)+(1-d)+e+(1-f))/6
def score9a(data):
    a,b,c,d,e,f,g,h,i = map(as_int, (data['9a.a'],data['9a.b'],data['9a.c'],data['9a.d'],data['9a.e'],data['9a.f'],data['9a.g'],data['9a.h'],data['9a.i']))
    return (a+(1-b)+c+d+(1-e)+f+(1-g)+(1-h)+(1-i))/9
def score9b(data):
    a,b,c,d,e,f = map(as_int, (data['9b.a'],data['9b.b'],data['9b.c'],data['9b.d'],data['9b.e'],data['9b.f']))
    return ((1-a)+(1-b)+(1-c)+(1-d)+e+(1-f))/6
def score10(data):
    a,b,c,d,e = map(as_int, (data['10.a'],data['10.b'],data['10.c'],data['10.d'],data['10.e']))
    return ((1-a)+(1-b)+c+d+e)/5

In [94]:
assessment = pd.DataFrame({"1a":score1a(everything),"1b":score1b(everything),"2a":score2a(everything),
                           "2b":score2b(everything),"2c":score2c(everything),"3":score3(everything),
                           "4a":score4a(everything),"5":score5(everything),"6a":score6a(everything),
                           "6b":score6b(everything),"7":score7(everything),"8":score8(everything),
                          "9a":score9a(everything),"9b":score9b(everything),"10":score10(everything)})

assessment.describe().loc[['mean','std']].round(2)

,1a,1b,2a,2b,2c,3,4a,5,6a,6b,7,8,9a,9b,10
mean,0.8,0.92,0.95,0.88,0.86,0.73,0.77,0.80,0.92,0.66,0.41,0.80,0.72,0.80,0.67
std,0.4,0.16,0.22,0.32,0.19,0.29,0.42,0.19,0.28,0.47,0.49,0.16,0.18,0.18,0.28


# Keras Computations

Begin by formatting our data before feeding it into a densely connected 3 layer network. 

In [95]:
train_size = 264

# Convert dataframe back to numpy array
assessement_np = assessment.to_numpy()

# Separate training data from testing data and grab labels
train_data = assessement_np[:train_size]
train_labels = everything['Final.total'].to_numpy()[:train_size].astype(np.float)

test_data = train_data
test_labels = train_labels

#test_data = assessement_np[train_size:]
#test_labels = everything['Final.total'].to_numpy()[train_size:].astype(np.float)

In [96]:
# Import Keras
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Build the model.
model = Sequential([
  Dense(1, activation='relu', input_shape=(15,))
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='mean_absolute_error',
  metrics=['mean_absolute_error'],
)

# Train the model.
model.fit(
  train_data,
  train_labels,
  epochs=1000,
  batch_size=32,
)

#Evaluate the model.
model.evaluate(
    test_data,
    test_labels
)

# Save the model to disk.
#model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

# Predict on the first 5 test images.
#predictions = model.predict(test_images[:5])

# Print our model's predictions.
#print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
#print(test_labels[:5]) # [7, 2, 1, 0, 4]

Epoch 1/1000
9/9 [==============================] - 0s 428us/step - loss: 0.3687 - mean_absolute_error: 0.3687
Epoch 2/1000
9/9 [==============================] - 0s 486us/step - loss: 0.2943 - mean_absolute_error: 0.2943
Epoch 3/1000
9/9 [==============================] - 0s 545us/step - loss: 0.2393 - mean_absolute_error: 0.2393
Epoch 4/1000
9/9 [==============================] - 0s 436us/step - loss: 0.2117 - mean_absolute_error: 0.2117
Epoch 5/1000
9/9 [==============================] - 0s 404us/step - loss: 0.2085 - mean_absolute_error: 0.2085
Epoch 6/1000
9/9 [==============================] - 0s 615us/step - loss: 0.2097 - mean_absolute_error: 0.2097
Epoch 7/1000
9/9 [==============================] - 0s 420us/step - loss: 0.2086 - mean_absolute_error: 0.2086
Epoch 8/1000
9/9 [==============================] - 0s 748us/step - loss: 0.2070 - mean_absolute_error: 0.2070
Epoch 9/1000
9/9 [==============================] - 0s 583us/step - loss: 0.2056 - mean_absolute_error: 0.2056
E

9/9 [==============================] - 0s 433us/step - loss: 0.1580 - mean_absolute_error: 0.1580
Epoch 75/1000
9/9 [==============================] - 0s 483us/step - loss: 0.1573 - mean_absolute_error: 0.1573
Epoch 76/1000
9/9 [==============================] - 0s 404us/step - loss: 0.1575 - mean_absolute_error: 0.1575
Epoch 77/1000
9/9 [==============================] - 0s 411us/step - loss: 0.1571 - mean_absolute_error: 0.1571
Epoch 78/1000
9/9 [==============================] - 0s 443us/step - loss: 0.1564 - mean_absolute_error: 0.1564
Epoch 79/1000
9/9 [==============================] - 0s 396us/step - loss: 0.1560 - mean_absolute_error: 0.1560
Epoch 80/1000
9/9 [==============================] - 0s 386us/step - loss: 0.1553 - mean_absolute_error: 0.1553
Epoch 81/1000
9/9 [==============================] - 0s 437us/step - loss: 0.1548 - mean_absolute_error: 0.1548
Epoch 82/1000
9/9 [==============================] - 0s 455us/step - loss: 0.1545 - mean_absolute_error: 0.1545
Epoch 

9/9 [==============================] - 0s 403us/step - loss: 0.1385 - mean_absolute_error: 0.1385
Epoch 146/1000
9/9 [==============================] - 0s 558us/step - loss: 0.1386 - mean_absolute_error: 0.1386
Epoch 147/1000
9/9 [==============================] - 0s 460us/step - loss: 0.1383 - mean_absolute_error: 0.1383
Epoch 148/1000
9/9 [==============================] - 0s 512us/step - loss: 0.1377 - mean_absolute_error: 0.1377
Epoch 149/1000
9/9 [==============================] - 0s 409us/step - loss: 0.1374 - mean_absolute_error: 0.1374
Epoch 150/1000
9/9 [==============================] - 0s 460us/step - loss: 0.1374 - mean_absolute_error: 0.1374
Epoch 151/1000
9/9 [==============================] - 0s 440us/step - loss: 0.1383 - mean_absolute_error: 0.1383
Epoch 152/1000
9/9 [==============================] - 0s 460us/step - loss: 0.1377 - mean_absolute_error: 0.1377
Epoch 153/1000
9/9 [==============================] - 0s 437us/step - loss: 0.1377 - mean_absolute_error: 0.137

9/9 [==============================] - 0s 414us/step - loss: 0.1289 - mean_absolute_error: 0.1289
Epoch 218/1000
9/9 [==============================] - 0s 453us/step - loss: 0.1287 - mean_absolute_error: 0.1287
Epoch 219/1000
9/9 [==============================] - 0s 386us/step - loss: 0.1286 - mean_absolute_error: 0.1286
Epoch 220/1000
9/9 [==============================] - 0s 408us/step - loss: 0.1288 - mean_absolute_error: 0.1288
Epoch 221/1000
9/9 [==============================] - 0s 409us/step - loss: 0.1291 - mean_absolute_error: 0.1291
Epoch 222/1000
9/9 [==============================] - 0s 415us/step - loss: 0.1283 - mean_absolute_error: 0.1283
Epoch 223/1000
9/9 [==============================] - 0s 418us/step - loss: 0.1284 - mean_absolute_error: 0.1284
Epoch 224/1000
9/9 [==============================] - 0s 446us/step - loss: 0.1291 - mean_absolute_error: 0.1291
Epoch 225/1000
9/9 [==============================] - 0s 381us/step - loss: 0.1280 - mean_absolute_error: 0.128

9/9 [==============================] - 0s 397us/step - loss: 0.1238 - mean_absolute_error: 0.1238
Epoch 290/1000
9/9 [==============================] - 0s 392us/step - loss: 0.1239 - mean_absolute_error: 0.1239
Epoch 291/1000
9/9 [==============================] - 0s 416us/step - loss: 0.1237 - mean_absolute_error: 0.1237
Epoch 292/1000
9/9 [==============================] - ETA: 0s - loss: 0.1335 - mean_absolute_error: 0.133 - 0s 420us/step - loss: 0.1235 - mean_absolute_error: 0.1235
Epoch 293/1000
9/9 [==============================] - 0s 430us/step - loss: 0.1240 - mean_absolute_error: 0.1240
Epoch 294/1000
9/9 [==============================] - 0s 403us/step - loss: 0.1238 - mean_absolute_error: 0.1238
Epoch 295/1000
9/9 [==============================] - 0s 411us/step - loss: 0.1240 - mean_absolute_error: 0.1240
Epoch 296/1000
9/9 [==============================] - 0s 406us/step - loss: 0.1236 - mean_absolute_error: 0.1236
Epoch 297/1000
9/9 [==============================] - 0s 

...

9/9 [==============================] - 0s 413us/step - loss: 0.1184 - mean_absolute_error: 0.1184
Epoch 722/1000
9/9 [==============================] - 0s 416us/step - loss: 0.1189 - mean_absolute_error: 0.1189
Epoch 723/1000
9/9 [==============================] - 0s 429us/step - loss: 0.1186 - mean_absolute_error: 0.1186
Epoch 724/1000
9/9 [==============================] - 0s 442us/step - loss: 0.1188 - mean_absolute_error: 0.1188
Epoch 725/1000
9/9 [==============================] - 0s 421us/step - loss: 0.1189 - mean_absolute_error: 0.1189
Epoch 726/1000
9/9 [==============================] - 0s 441us/step - loss: 0.1189 - mean_absolute_error: 0.1189
Epoch 727/1000
9/9 [==============================] - 0s 421us/step - loss: 0.1186 - mean_absolute_error: 0.1186
Epoch 728/1000
9/9 [==============================] - 0s 412us/step - loss: 0.1192 - mean_absolute_error: 0.1192
Epoch 729/1000
9/9 [==============================] - 0s 374us/step - loss: 0.1188 - mean_absolute_error: 0.118

9/9 [==============================] - 0s 389us/step - loss: 0.1190 - mean_absolute_error: 0.1190
Epoch 794/1000
9/9 [==============================] - 0s 434us/step - loss: 0.1188 - mean_absolute_error: 0.1188
Epoch 795/1000
9/9 [==============================] - 0s 420us/step - loss: 0.1190 - mean_absolute_error: 0.1190
Epoch 796/1000
9/9 [==============================] - 0s 441us/step - loss: 0.1189 - mean_absolute_error: 0.1189
Epoch 797/1000
9/9 [==============================] - 0s 451us/step - loss: 0.1190 - mean_absolute_error: 0.1190
Epoch 798/1000
9/9 [==============================] - ETA: 0s - loss: 0.1395 - mean_absolute_error: 0.139 - 0s 534us/step - loss: 0.1189 - mean_absolute_error: 0.1189
Epoch 799/1000
9/9 [==============================] - 0s 451us/step - loss: 0.1186 - mean_absolute_error: 0.1186
Epoch 800/1000
9/9 [==============================] - 0s 466us/step - loss: 0.1188 - mean_absolute_error: 0.1188
Epoch 801/1000
9/9 [==============================] - 0s 

9/9 [==============================] - 0s 410us/step - loss: 0.1188 - mean_absolute_error: 0.1188
Epoch 865/1000
9/9 [==============================] - 0s 431us/step - loss: 0.1189 - mean_absolute_error: 0.1189
Epoch 866/1000
9/9 [==============================] - 0s 422us/step - loss: 0.1188 - mean_absolute_error: 0.1188
Epoch 867/1000
9/9 [==============================] - 0s 409us/step - loss: 0.1200 - mean_absolute_error: 0.1200
Epoch 868/1000
9/9 [==============================] - 0s 393us/step - loss: 0.1197 - mean_absolute_error: 0.1197
Epoch 869/1000
9/9 [==============================] - 0s 439us/step - loss: 0.1185 - mean_absolute_error: 0.1185
Epoch 870/1000
9/9 [==============================] - 0s 386us/step - loss: 0.1188 - mean_absolute_error: 0.1188
Epoch 871/1000
9/9 [==============================] - 0s 411us/step - loss: 0.1190 - mean_absolute_error: 0.1190
Epoch 872/1000
9/9 [==============================] - 0s 403us/step - loss: 0.1187 - mean_absolute_error: 0.118

9/9 [==============================] - 0s 464us/step - loss: 0.1195 - mean_absolute_error: 0.1195
Epoch 937/1000
9/9 [==============================] - 0s 452us/step - loss: 0.1193 - mean_absolute_error: 0.1193
Epoch 938/1000
9/9 [==============================] - 0s 439us/step - loss: 0.1189 - mean_absolute_error: 0.1189
Epoch 939/1000
9/9 [==============================] - 0s 429us/step - loss: 0.1192 - mean_absolute_error: 0.1192
Epoch 940/1000
9/9 [==============================] - 0s 410us/step - loss: 0.1186 - mean_absolute_error: 0.1186
Epoch 941/1000
9/9 [==============================] - 0s 460us/step - loss: 0.1188 - mean_absolute_error: 0.1188
Epoch 942/1000
9/9 [==============================] - 0s 460us/step - loss: 0.1197 - mean_absolute_error: 0.1197
Epoch 943/1000
9/9 [==============================] - 0s 417us/step - loss: 0.1186 - mean_absolute_error: 0.1186
Epoch 944/1000
9/9 [==============================] - 0s 436us/step - loss: 0.1187 - mean_absolute_error: 0.118

[0.11857003718614578, 0.11857003718614578]

In [82]:
# Check our predications against the true values
test_data = assessement_np[21:25]
model.predict(test_data)

array([[0.8746185 ],
       [0.80817014],
       [0.84754777],
       [0.6865788 ]], dtype=float32)

In [83]:
print(everything['Final.total'].to_numpy()[21:25].astype(np.float))

[0.76 0.74 0.86 0.77]


In [90]:
model.layers[0].get_weights()[0]

array([[ 0.05184653],
       [ 0.06375918],
       [-0.04522215],
       [ 0.07846978],
       [ 0.12378085],
       [ 0.05643027],
       [ 0.0386402 ],
       [ 0.04811145],
       [-0.05433674],
       [ 0.06456778],
       [ 0.09308463],
       [ 0.1739876 ],
       [ 0.15899083],
       [ 0.00432104],
       [ 0.09174275]], dtype=float32)